In [ ]:
import pandas as pd
import json
import random

In [ ]:
#protocol parameters

name = input("protocol name: ")
filename = input("file name: ") + ".json"
parameters = input("parameters: ")


RT_protocol = 'rt_protocol_example.csv' 
standard_protocol = 'protocol_example.csv'

In [ ]:
#generate protocol functions
def generate_standard_protocol(input_csv, filename=filename):
    # Read CSV into a DataFrame
    df = pd.read_csv(input_csv)

    # Initialize the JSON structure
    json_data = {
        "name": name,
        "parameters": [parameters],
        "active": True,
        "classes": [],
        "randomClassOrder": False
    }

    # Iterate through rows in the CSV
    for index, row in df.iterrows():
        # Extract relevant information from the CSV row
        class_name = row['class_name']
        inactive_duration = row['inactive_duration']
        active_duration = row['active_duration']
        preamble = row['preamble']
        epilogue = row['epilogue']
        activeClass = row['activeClass_name']
        repetitions = int(row['repetitions'])

        # Create the class structure
        class_structure = {
            "class": class_name,
            "intervals": [
                {"label": "inactive", "durationInMillis": inactive_duration},
                {"label": activeClass, "durationInMillis": active_duration},
                {"label": "inactive", "durationInMillis": inactive_duration}
            ],
            "repetitions": repetitions,
            "preamble": preamble,
            "epilogue": epilogue
        }

        # Append the class structure to the classes list
        json_data["classes"].append(class_structure)

    # Write the JSON data to a file
    with open(output_json, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)




def generate_RT(csv_file, filename=filename):
    df = pd.read_csv(csv_file)
    # Iterate through rows in the CSV
    for index, row in df.iterrows():
        # Extract relevant information from the CSV row
        total_duration = int(row['total_duration'])
        stimulus_duration = int(row['stimulus_duration'])
        max_isi = int(row['max_isi'])
        min_isi = int(row['min_isi'])
        r_led = int(row['r_led'])
        g_led = int(row['g_led'])
        b_led = int(row['b_led'])
        led_color = [r_led, g_led, b_led]
        intensity = int(row['intensity'])

    #generate random isi 
    num_intervals = int(total_duration / (stimulus_duration + (max_isi + min_isi) / 2))
    intervals = []
    for _ in range(num_intervals):
        isi = random.randint(min_isi, max_isi)
        intervals.append(isi)

    data = {
        "name": name,
        "parameters": ["test"],
        "active": True,
        "classes": [
            {
                "class": "PMT",
                "intervals": [],
                "type": "REACTION_TIME"
            }
        ],
        "randomClassOrder": True
    }

    class_data = data["classes"][0]

    for i, duration in enumerate(intervals):
        # Unlabeled interval
        class_data["intervals"].append({
            "durationInMillis": duration,
            "ledOptions": {
                "enabled": False
            }
        })

        # STIM interval
        class_data["intervals"].append({
            "label": "STIM",
            "durationInMillis": stimulus_duration,
            "ledOptions": {
                "color": {
                    "red": led_color[0],
                    "green": led_color[1],
                    "blue": led_color[2]
                },
                "intensity": intensity,
                "enabled": True
            }
        })

    # Set the last interval as an unlabeled interval if the total count is odd
    if len(intervals) % 2 == 1:
        class_data["intervals"].pop()

    with open(filename, "w") as file:
        json.dump(data, file, indent=2)